<h1>DataSciencBowl2019:Tuning LightGBM Parameter Using OPTUNA</h1>
<h4>
In this kernel, I will be emphasizing upon how to tune LightGBM Parameters using OPTUNA. Hyperparameter tuning is a very tedious & time consuming task.There are multiple strategies available to find optimal parameters using grid-search & random-search. OPTUNA can be used to get a good choice of Hyperparameters in a smarter way. OPTUNA is a hyperparametes optimzation framework based on Bayesian methods & to know more about it, please visit https://optuna.org/. I used OPTUNA to tune parameters for my LightGBM model for DataScienceBowl2019 Competition. To know more about DataScienceBowl2019, please visit https://datasciencebowl.com/ </h4>

**<h2>1. Importing Libraries & Loading the Data</h2>**   <h4>This notebook was used as a submission to DataScienceBowl 2019 Competition. Since this is a kernel emphasizing upon Hyperparameter tuning using OPTUNA, I will be skipping the data processing & feature engineering steps. </h4>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Any results you write to the current directory are saved as output.
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold, StratifiedKFold
import gc
import optuna
pd.set_option('display.max_columns', 1000)

def read_data():
    sample_submission = pd.read_csv("../input/data-science-bowl-2019/sample_submission.csv")
    specs = pd.read_csv("../input/data-science-bowl-2019/specs.csv")
    test = pd.read_csv("../input/data-science-bowl-2019/test.csv")
    train = pd.read_csv("../input/data-science-bowl-2019/train.csv")
    train_labels = pd.read_csv("../input/data-science-bowl-2019/train_labels.csv") 
    return train, test, train_labels, specs, sample_submission

def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    # convert text into datetime
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code

# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
                    
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features.update(last_accuracy_title.copy())
            
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort = False)), total = 17000):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort = False), total = 1000):
        test_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    categoricals = ['session_title']
    return reduce_train, reduce_test, categoricals

In [ ]:
print('********Reading & Processing Data**********')
# read data
train, test, train_labels, specs, sample_submission = read_data()
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(train, test, train_labels)
# tranform function to get the train and test set
reduce_train, reduce_test, categoricals = get_train_and_test(train, test)

reduce_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_train.columns]
reduce_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_test.columns]

features = reduce_train.loc[(reduce_train.sum(axis=1) != 0), (reduce_train.sum(axis=0) != 0)].columns # delete useless columns
features = [x for x in features if x not in ['accuracy_group', 'installation_id']]

target = 'accuracy_group'
xTrain, xTest = reduce_train[features],reduce_test[features]
yTrain, yTest = reduce_train[target],reduce_test[target]
categoryCols = ['session_title']
print('******** Finished Reading & Preparing the Data**********')

**<h1>2.OPTUNA</h1>** <h4>Optuna uses Bayesian methods to figure out an optimal set of hyperparameters. For more information on Bayesian methods for searching optimal parameters, check out this wonderful article : https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f .</h4>

<h2> 2.1 Objective Function</h2> 
<h4>First important step is to define an objective function.Let's understand this with an example from OPTUNA docs.</h4>

<h4><mark>def objective(trial):</mark></h4>
<h4><mark>x = trial.suggest_uniform('x', -10, 10)</mark></h4>
<h4><mark>return (x - 2) ** 2</mark></h4>

<h4>This function returns the value of (𝑥 − 2)^2.Our goal is to find the value of x that minimizes the output of the objective function.This is the optimization.During the optimization, Optuna repeatedly calls and evaluates the objective function with different values of x.A Trial object corresponds to a single execution of the objective function and is internally instantiated upon each invocation of the function.</h4>  <h4>In our case, we will be training lightGBM Model and using the cross validation score for evaluation.We will be returing this score from our objective function. For Data Science Bowl 2019 competition, the metric supposed to be used was cohen_kappa_score. Even though I will be computing cohen_kappa_score, but I will be using cross validation score for evaluation of the objective function for the sake of simplicity.</h4>

In [ ]:
from optuna import Trial

def objective(trial:Trial,fastCheck=True,targetMeter=0,returnInfo=False):
    folds = 5
    seed  = 666
    shuffle = False
    kf = KFold(n_splits=folds,shuffle=False,random_state=seed)
    yValidPredTotal = np.zeros(xTrain.shape[0])
    gc.collect()
    catFeatures=[xTrain.columns.get_loc(catCol) for catCol in categoryCols]
    models=[]
    validScore=0
    for trainIdx,validIdx in kf.split(xTrain,yTrain):
        trainData=xTrain.iloc[trainIdx,:],yTrain[trainIdx]
        validData=xTrain.iloc[validIdx,:],yTrain[validIdx]
        model,yPredValid,log = fitLGBM(trial,trainData,validData,catFeatures=categoryCols,numRounds=1000)
        yValidPredTotal[validIdx]=yPredValid
        models.append(model)
        gc.collect()
        validScore+=log["validRMSE"]
    validScore/=len(models)
    return validScore

<h2>2.2 Defining Parameter Space for OPTUNA </h2>
<h4>OPTUNA supports five kind of parameters</h4>

<ul>
<h4>def objective(trial):</h4>
<h4><mark>#Categorical parameter : A categorical distribution</mark></h4>
<h4>optimizer = trial.suggest_categorical('optimizer', ['MomentumSGD', 'Adam'])</h4>
<h4><mark>#Int parameter : A uniform distribution on integers.</mark></h4>
<h4>num_layers = trial.suggest_int('num_layers', 1, 3)</h4>
<h4><mark>#Uniform Parameter : A uniform distribution in the linear domain.</mark></h4>
<h4>dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)</h4>
<h4><mark>#Loguniform Parameter : A uniform distribution in the log domain.</mark></h4>
<h4>learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)</h4>
<h4><mark>#Discrete-uniform parameter : A discretized uniform distribution in the linear domain.</mark></h4>
<h4>drop_path_rate = trial.suggest_discrete_uniform('drop_path_rate', 0.0, 1.0, 0.1)</h4>
</ul>
<h4>In the below function, we will be setting up the parameter space for LightGBM. Please refer to LightGBM Documentation for exploring more on the LightGBM parameters https://lightgbm.readthedocs.io/en/latest/Parameters.html. Below parameter space may not be exhaustive one and can be modified as per the requirement</h4>


In [ ]:
def fitLGBM(trial,train,val,catFeatures=None,numRounds=1500):
    xTrainLGBM,yTrainLGBM = train
    xValidLGBM,yValidLGBM = val
    boosting_list = ['gbdt','goss']
    objective_list_reg = ['huber', 'gamma', 'fair', 'tweedie']
    objective_list_class = ['binary', 'cross_entropy']
    params={
      'boosting':trial.suggest_categorical('boosting',boosting_list),
      'num_leaves':trial.suggest_int('num_leaves', 2, 2**11),
      'max_depth':trial.suggest_int('max_depth', 2, 25),
      'max_bin': trial.suggest_int('max_bin', 32, 255),      
      'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 256),
      'min_data_in_bin': trial.suggest_int('min_data_in_bin', 1, 256),
      'min_gain_to_split' : trial.suggest_discrete_uniform('min_gain_to_split', 0.1, 5, 0.01),      
      'lambda_l1':trial.suggest_loguniform('lamda_l1',1e-8,10),
      'lambda_l2':trial.suggest_loguniform('lamda_l2',1e-8,10),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
      'metric':trial.suggest_categorical('metric', ['RMSE']),
      'objective':trial.suggest_categorical('objective',objective_list_reg),
      'bagging_fraction':trial.suggest_discrete_uniform('bagging_fraction',0.5, 1, 0.01),
      'feature_fraction':trial.suggest_discrete_uniform('feature_fraction',0.5, 1, 0.01),
    }
    earlyStop=20
    verboseEval=0
    dTrain = lgb.Dataset(xTrainLGBM,label=yTrainLGBM,categorical_feature=catFeatures)
    dValid = lgb.Dataset(xValidLGBM,label=yValidLGBM,categorical_feature=catFeatures)
    watchlist = [dTrain,dValid]

    # Callback for pruning.
    lgbmPruningCallback = optuna.integration.LightGBMPruningCallback(trial, 'rmse', valid_name='valid_1')

    model = lgb.train(params,train_set=dTrain,num_boost_round=numRounds,valid_sets=watchlist,verbose_eval=verboseEval,early_stopping_rounds=earlyStop,callbacks=[lgbmPruningCallback])

    #predictions
    pred_val=model.predict(xValidLGBM,num_iteration=model.best_iteration)
    pred_val[pred_val <= 1.12232214] = 0
    pred_val[np.where(np.logical_and(pred_val > 1.12232214, pred_val <= 1.73925866))] = 1
    pred_val[np.where(np.logical_and(pred_val > 1.73925866, pred_val <= 2.22506454))] = 2
    pred_val[pred_val > 2.22506454] = 3
    oofPred = pred_val.astype(int)        
    score=cohen_kappa_score(oofPred,yValidLGBM,weights='quadratic')
    print('***********************choen_kappa_score :',score)
    log={'trainRMSE':model.best_score['training']['rmse'],
       'validRMSE':model.best_score['valid_1']['rmse']}
    return model,pred_val,log

<h2>2.3 Optimization & STUDY Object </h2>
**Below are some important terminologies mentioned in the OPTUNA docs, understanding which will make our jobs easier:**
* **Trial**: **A single call of the objective function**
* **Study**: **An optimization session, which is a set of trials**
* **Parameter**: **A variable whose value is to be optimized, such as x in the above example**  

<h4>
I have already explained about Trial & Parameter in the above sections. In Optuna, we use the study object to manage optimization. Method create_study() returns a study object. A study object has useful properties for analyzing the optimization outcome.Once we create the study object, we can call the Optimize() and let the show begin!
</h4>

In [ ]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective,n_trials=10)#For the sake of simplicity, I have kept n_trials as less, but this can be altered for better results

<h2>2.4 Pruning Uncompromising Trials in OPTUNA </h2>
<h4>
Unpromising trials can be automotically detected at the early stages of the training & thus can be preempted(a.k.a., automated early-stopping).Optuna provides interfaces to concisely implement the pruning mechanism in iterative training algorithms.To implement pruning mechanism in much simpler forms, Optuna provides integration modules for the following libraries
</h4>
* **XGBoost**: optuna.integration.XGBoostPruningCallback
* **LightGBM**: optuna.integration.LightGBMPruningCallback
* **Chainer**: optuna.integration.ChainerPruningExtension
* **Keras**: optuna.integration.KerasPruningCallback
* **TensorFlow** optuna.integration.TensorFlowPruningHook
* **tf.keras** optuna.integration.TFKerasPruningCallback
* **MXNet** optuna.integration.MXNetPruningCallback
* **PyTorch** Ignite optuna.integration.PyTorchIgnitePruningHandler
* **PyTorch** Lightning optuna.integration.PyTorchLightningPruningCallback
* **FastAI** optuna.integration.FastAIPruningCallback

**In the function fitLGBM defined above, I have added a Pruning Callback function:  
lgbmPruningCallback = optuna.integration.LightGBMPruningCallback(trial, 'rmse', valid_name='valid_1')**

<h2>2.5 OPTUNA Study History : Analysis & Visualization </h2>
<h3>We can get the entire history of all the trials by calling *study.trials_dataframe()* which returns a pandas.DataFrame</h3>

In [ ]:
df = study.trials_dataframe()
df

<h3>We can obtain best trial parameters by calling study.best_trial</h3>

In [ ]:
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

<h3>optuna.visualization.plot_optimization_history(study) : This function plots optimization history of all trials in a study</h3>

In [ ]:
optuna.visualization.plot_optimization_history(study)

<h3>optuna.visualization.plot_slice(study, params=None) : This function plots the parameter relationship as slice plot in a study</h3>

In [ ]:
optuna.visualization.plot_slice(study)

<h2>optuna.visualization.plot_parallel_coordinate(study, params=None) : This function plots the high-dimentional parameter relationships in a study.</h2>

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

<h2>
That's all from my side. Hope you find this kernel useful and please don't forget to UPVOTE this kernel. CHEERS!!
</h2>